In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import os

In [14]:
# csv paths
trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
new_trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samara_trips_2025.xlsx"
trip_data = pd.read_csv(trip_csv)
end_path = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\generated data\samsara_wait_times.xlsx"

In [24]:
# group by data and location with ascending startMs
sort_by_truck = trip_data.sort_values(by=["start_time"], ascending=True)

In [37]:
%%time
# csv paths
trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
# trip_data = pd.read_csv(trip_csv)
trip_data = pd.read_excel(new_trip_csv)
trip_data['wait_time'] = ""
# make a list of dsates and one of vins
date_list = trip_data['Date'].unique()
# date_list = ['01/02/2025']
# vin_list = ['1GT19LE79PF224344']
vin_list = trip_data['Vin Number'].unique()
df_list = []
for vin in vin_list:
    vin_df = trip_data[trip_data['Vin Number']==vin]
    for day in date_list:
        date_subset = vin_df[vin_df['Date'] == day]
        date_subset = date_subset.reset_index()
        # date_subset['wait_time'] = ""
        # make sure that this is still in the right order to calculate
        date_subset = date_subset.sort_values(by='start_time')
        df_length = len(date_subset)
        if df_length > 1:
            for row in range(0,df_length-1):
                current = date_subset.iloc[row]
                next_row = date_subset.iloc[row+1]
                if current['endLocation'] == next_row['startLocation']:
                    wait_time = pd.to_datetime(next_row['start_time']) - pd.to_datetime(current['end_time'])
                    date_subset.loc[row, 'wait_time'] = wait_time.total_seconds() / 60
            df_list.append(date_subset)

full_df = pd.concat(df_list)  

CPU times: total: 22.3 s
Wall time: 22.6 s


In [38]:
# Aggregate data
no_blank_times = full_df[full_df['wait_time'] != ""]
# grouped = 
no_blank_times.shape

SyntaxError: invalid syntax (1784908206.py, line 3)

In [39]:
# create excel for spot checks
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
full_df.to_excel(writer, sheet_name="full_df")
writer.close()

In [23]:
full_df

,index,CalendarSKey,id,startMs,endMs,startLocation,endLocation,distanceMeters,fuelConsumedMl,tollMeters,...,Date,Miles,MPG,Hours,UT%,EquipmentID,Vin Number,HRRef,Tags,wait_time
0,0,2025010202,1885347,1735861424030,1735865196019,"East County Road 14, Johnstown, CO","Vance Street, Lakewood, CO",79078,12904,1506,...,2025-01-02,49.136913,14.414396,1.050000,0.087500,PU.4344,1GT19LE79PF224344,NaN,NaN,
1,467,2025010202,1885349,1735867075025,1735867400007,"West 1st Avenue, Denver, CO","South King Street, Denver, CO",922,377,0,...,2025-01-02,0.572906,5.752474,0.100000,0.008333,PU.4344,1GT19LE79PF224344,NaN,NaN,
2,471,2025010202,1885348,1735866301089,1735866745025,"Vance Street, Lakewood, CO","West 1st Avenue, Denver, CO",4189,630,0,...,2025-01-02,2.602930,15.639941,0.116666,0.010000,PU.4344,1GT19LE79PF224344,NaN,NaN,-735.7
3,36844,2025010202,1885336,1735822603023,1735825493030,"West 1st Avenue, Denver, CO","East County Road 14, Johnstown, CO",75153,11167,2594,...,2025-01-02,46.698025,15.829782,0.800000,0.066667,PU.4344,1GT19LE79PF224344,NaN,NaN,42.6
4,36849,2025010202,1885337,1735828049039,1735831622024,"East County Road 14, Johnstown, CO","East County Road 14, Johnstown, CO",75713,11298,0,...,2025-01-02,47.045994,15.762823,1.000000,0.083333,PU.4344,1GT19LE79PF224344,NaN,NaN,182.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,36666,2025010202,1885155,1735854845204,1735856444018,"East 88th Avenue, Commerce City, CO","Valley Head Street, Denver, CO",17698,8000,0,...,2025-01-02,10.997055,5.203545,0.433333,0.035833,TT.5370,5KJJBPDV1RLUR5370,1774.0,"Truck Drivers Pay Roll, TT DRIVERS ...",6.95
8,36667,2025010202,1885156,1735856861011,1735859887020,"Valley Head Street, Denver, CO","Valley Head Street, Denver, CO",32522,16000,145,...,2025-01-02,20.208284,4.781040,0.850000,0.070833,TT.5370,5KJJBPDV1RLUR5370,1774.0,"Truck Drivers Pay Roll, TT DRIVERS ...",7.95
9,36668,2025010202,1885157,1735860364016,1735863935055,"Valley Head Street, Denver, CO","West 62nd Avenue, 1.2 mi SSE Twin Lakes, Adams...",44354,19500,0,...,2025-01-02,27.560366,5.350117,0.983333,0.081667,TT.5370,5KJJBPDV1RLUR5370,1774.0,"Truck Drivers Pay Roll, TT DRIVERS ...",7.75
10,36672,2025010202,1885158,1735864400032,1735864417025,"West 62nd Avenue, 1.2 mi SSE Twin Lakes, Adams...","Huron Street, 1.2 mi SSE Twin Lakes, Adams Cou...",31,0,0,...,2025-01-02,0.019262,NaN,0.000000,0.000000,TT.5370,5KJJBPDV1RLUR5370,1774.0,"Truck Drivers Pay Roll, TT DRIVERS ...",6.133333


In [7]:
len(full_df[full_df['wait_time'] != ""])

35029

In [8]:
delta = 49492 - 35029 
delta

14463